# Creacion y prueba de la implementacion del algoritmo de descenso de gradiente

Se debe probara el algoritmo usando un dataset de 3 columnas y 1000 filas para las cuales se aplicara regresion sobre una de las variables generando una funcion de regresion de la forma $f(x) = w_1 x_1 + w_2 x_2 + w_0$.

In [5]:
import pandas as pd
import numpy as np

In [4]:
training_df = pd.DataFrame(columns=['var_a', 'var_b', 'var_c'])
training_df

,var_a,var_b,var_c


La funcion ``` np.random.rand() ``` genera numeros aleatorios entre [0, 1) de una poblacion con distribucion uniforme

In [91]:
n_values = 1000
training_df['var_a'] = np.random.rand(n_values)
training_df['var_b'] = np.random.rand(n_values)
training_df['var_c'] = np.random.rand(n_values)
training_df

,var_a,var_b,var_c
0,0.254110,0.425389,0.011332
1,0.253649,0.689973,0.814003
2,0.187070,0.116594,0.962726
3,0.617361,0.158843,0.345723
4,0.539590,0.641035,0.087533
...,...,...,...
995,0.223238,0.391551,0.675875
996,0.368537,0.094390,0.898275
997,0.760945,0.277484,0.422242
998,0.551023,0.886850,0.907689
